# 6 Dimension KPI 2023

In [1]:
import cx_Oracle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import configparser

## Parameter

In [2]:
as_week = 2023025
raw_dir = f"..\..\data\\raw"
op_dir = f"..\..\data\output"
raw_file_name = f'Raw-Agg Perf_Wk_Area {as_week}'
op_file_name = f'{raw_file_name.replace("Raw-Agg", "BG")}'

In [3]:
config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

username = config['TDMDBPR']['username']
password = config['TDMDBPR']['password']
db = config['TDMDBPR']['db']
host = config['TDMDBPR']['host']
port = config['TDMDBPR']['port']

## Import Business Growth data

In [4]:
conStr = f'{username}/{password}@{host}:{port}/{db}'
conn = cx_Oracle.connect(conStr)
print(f'{db} : Connected')
cur = conn.cursor()

v_week = (as_week, )


try:
    print(f'\nExecute query as Week {v_week[0]}...')

    sqlTxt = """
        WITH W_VAR (AS_OF_WK) AS 
        ( 
            SELECT :1 AS AS_OF_WK FROM DUAL 
        ) 
        -----------------------------------------------------------------------------------------------------------------------

        SELECT *
        FROM ( 
            SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, DAY_DESC, MONTH_DESC, QUARTER_DESC, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME, DIMENSION_KPI_FLAG, BG_FLAG--, METRIC_NAME_GROUP
                , ARAE_NO, AREA_TYPE, AREA_CD, AREA_NAME
                , AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD 
                , ACTUAL_SNAP, ACTUAL_AGG, TARGET_SNAP, TARGET_AGG, PPN_TM
            FROM ( 
                SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, DAY_DESC, MONTH_DESC, QUARTER_DESC, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME 
                    , CASE 	WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 ELSE 0 END ARAE_NO
                    , AREA_TYPE, AREA_CD, AREA_NAME
                    , CASE 	WHEN ( METRIC_CD IN ('DNEC10006', 'TNEC10002', 'TNEC20002') --NETWORK EXPERIENCE 
                                OR METRIC_CD IN ('B1R000100', 'B2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00142') --BUSINESS GROWTH 
                                OR METRIC_CD IN ('DSER21101', 'DSER22101', 'TSER21101', 'TSER22101', 'TSER23101', 'DB2S000200', 'DB1S000200', 'TB2S000200', 'TB1S000200', 'TSER13100', 'TSER14100') --CUSTOMER RETENTION & EXPERIENCE 
                                OR METRIC_CD IN ('TNSC00011', 'TNSC00013', 'TNSC00087', 'TNSC00101', 'TNSC00144', 'TNSC00020', 'TNSC00130') --NEW S-CURVE 
                                --COST & PRODUCTIVITY (Unknown ?) 
                                OR METRIC_CD IN ('DIS10007', 'DIS10011', 'DIS10001') --DIGITALIZATION 
                            ) THEN 'Y' ELSE 'N' END DIMENSION_KPI_FLAG 
                    , CASE 	WHEN METRIC_CD IN ('B1R000100', 'B2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00142') --BUSINESS GROWTH 
                            THEN 'Y' ELSE 'N' END BG_FLAG 
                    , METRIC_NAME_GROUP, PPN_TM, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, ACTUAL_AGG, TARGET_SNAP, TARGET_AGG
                FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK 
                WHERE TM_KEY_WK <= (SELECT AS_OF_WK FROM W_VAR)
                AND AREA_TYPE IN ('P', 'G', 'H')
            ) A 
        ) B 
        WHERE METRIC_CD IN ('B1R000100', 'B2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00142')
    """
    
    cur.execute(sqlTxt, v_week)

    print(f'\nDone !!!')

    rows = cur.fetchall()
    
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])


except cx_Oracle.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{db} : Disconnected')

TDMDBPR : Connected

Execute query as Week 2023025...

Done !!!

TDMDBPR : Disconnected


In [5]:
df.shape

(67860, 30)

In [6]:
df.tail(3)

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,DAY_DESC,MONTH_DESC,QUARTER_DESC,CENTER,METRIC_GRP,...,AGG_TYPE,RR_IND,GRY_IND,UOM,PERIOD,ACTUAL_SNAP,ACTUAL_AGG,TARGET_SNAP,TARGET_AGG,PPN_TM
67857,20230625,2023025,202306,20232,2023,Sunday,June,Quarter 2,Sales & Revenue,Revenue,...,S,0,GYR,baht,M,1187858.01,13207851.57,1.537517e+06,1.323853e+07,2023-06-29 10:06:31
67858,20230625,2023025,202306,20232,2023,Sunday,June,Quarter 2,Sales & Revenue,Revenue,...,S,0,GYR,baht,M,79941.97,799861.26,9.846239e+04,8.477937e+05,2023-06-29 10:06:31
67859,20230625,2023025,202306,20232,2023,Sunday,June,Quarter 2,Sales & Revenue,Revenue,...,S,0,GYR,baht,M,3190.00,252333.12,NaN,NaN,2023-06-29 10:06:31


### Generate Rawdata as CSV file

In [7]:
raw_file = f'{raw_dir}\{raw_file_name}'

df.to_csv(f'{raw_file}.csv', index=False, encoding='utf-8')

print(f'\n"{raw_file}.csv" is generated')


"..\..\data\raw\Raw-Agg Perf_Wk_Area 2023025.csv" is generated


## Aggregation

### Data Prep

In [8]:
data_src = f'{raw_file}.csv'
df_bg = pd.read_csv(data_src)
df_bg.head(3)

C:\Users\Narut4\AppData\Local\Temp\ipykernel_9528\655902469.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bg = pd.read_csv(data_src)


,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,DAY_DESC,MONTH_DESC,QUARTER_DESC,CENTER,METRIC_GRP,...,AGG_TYPE,RR_IND,GRY_IND,UOM,PERIOD,ACTUAL_SNAP,ACTUAL_AGG,TARGET_SNAP,TARGET_AGG,PPN_TM
0,20230301,2023009,202303,20231,2023,Wednesday,March,Quarter 1,Sales & Revenue,Revenue,...,S,0,GYR,baht,M,68778.3076,68778.3076,70927.3896,70927.3896,2023-06-29 10:06:31
1,20230301,2023009,202303,20231,2023,Wednesday,March,Quarter 1,Sales & Revenue,Revenue,...,S,0,GYR,baht,M,82900.2484,82900.2484,70927.3896,70927.3896,2023-06-29 10:06:31
2,20230301,2023009,202303,20231,2023,Wednesday,March,Quarter 1,Sales & Revenue,Revenue,...,S,0,GYR,baht,M,86156.4179,86156.4179,47615.4873,47615.4873,2023-06-29 10:06:31


In [9]:
df_bg.shape

(67860, 30)

In [10]:
df_bg['METRIC_NAME'].unique()

array(['Postpaid Revenue', 'Prepaid Revenue',
       'TDG Revenue (Performance View)', 'TOL Revenue', 'TVS Revenue'],
      dtype=object)